# Points à vérifier dans mon code: 

In [ ]:
# Trouver un moyen d'uniformiser 'time_step_per_hour' pour toute les données, genre dénominateur commun etc.
dates_verif_obj = DatesVerifFeatureVect(dataset.df_dates, Weeks = args.W, Days = args.D, historical_len = args.H, step_ahead = args.step_ahead, time_step_per_hour = dataset.time_step_per_hour)
dataset.get_feature_vect(invalid_dates)
# ========== .... ==========

# Mini Test pour voir si tout le pre-processing fonctionne: 

In [1]:

from DL_class import InvalidDatesCleaner,DatesVerifFeatureVect
from dataset import TrainValidTest_Split_Normalize

import os
import pandas as pd 
import torch
import pickle 
import numpy as np 

from paths import folder_path,file_name
from config import get_args
from utilities_DL import get_DataSet_and_invalid_dates, match_period_coverage_with_netmob


# ==================== Load Subway-in Dataset : ====================
# Load config
model_name = 'STGCN' #'CNN'
netmob = True
args = get_args(model_name)
#args = get_args(model_name = model_name,learn_graph_structure = True)  # MTGNN

# Modification : 
args.K_fold = 1
args.ray = False

# Load Init DataSet 
dataset,invalid_dates = get_DataSet_and_invalid_dates(args.abs_path, folder_path,file_name,
                                                      args.W,args.D,args.H,args.step_ahead,
                                                      single_station = False,coverage_period = None)

coverage = match_period_coverage_with_netmob(dataset)

dataset,invalid_dates = get_DataSet_and_invalid_dates(args.abs_path, folder_path,file_name,
                                                      args.W,args.D,args.H,args.step_ahead,
                                                      single_station = False,coverage_period = coverage)

# ==================== ........................ ====================


if torch.cuda.is_available():
    data_folder_path = '../../../data/' 
else:
    data_folder_path = '../../Data/'


# === INIT ====
save_folder = f"{data_folder_path}NetMob_tensor/"
netmob_data_folder_path = f"{data_folder_path}NetMob/"
step_south_north = 287  # Incremente by 287-ids when passing from south to north. 
epsilon=1000  #epsilon : radius, in meter (1000m) 
# W,H = 2*(epsilon//100 + 1), 2*(epsilon//100 + 1)

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
# === .... ===




# ===== Load NetMob Data: =====
# NetMob Tensor : [T,N,C,H,W]
# dims : [0,-2,-1]  -> dimension for which we want to retrieve stats 
try :
    netmob_T = torch.stack([torch.load(f"{save_folder}station_{station}.pt") for station in ref_subway.COD_TRG])
    netmob_T = netmob_T.permute(1,0,*range(2, netmob_T.dim()))

except:
    netmob_T = torch.randn(7300,40,2,22,22)
    print("Load des données NetMob .pt impossible. Création d'un random Tensor")

print('Init NetMob Dataset: ', netmob_T.size())
print('Number of Nan Value: ',torch.isnan(netmob_T).sum())
print('Total Number of Elements: ', netmob_T.numel() )
# ===== ....... =====
T = netmob_T.size(0)

# Tackle a specific fold : 
netmob_T1 = netmob_T[:100]

# Init :
dims = [0,3,4]#[0,-2,-1]
minmaxnorm = True
standardize = False

# ============ Load Train/Valid/Test Indices and removed forbidden dates : ============
# invalid dates = 
# invalid_indices = get_indices_from_dates(invalid)
indices = np.arange(T)
np.random.shuffle(indices)
invalid_indices = indices[:100]

# Get Split indices :
train_indices = np.arange(50)
valid_indices = np.arange(60,70)
test_indices = np.arange(80,100)

# Remove invalid_dates from indices :
cleaner = InvalidDatesCleaner(invalid_indices = invalid_indices)

train_indices = cleaner.clean_indices(train_indices)
valid_indices = cleaner.clean_indices(valid_indices)
test_indices = cleaner.clean_indices(test_indices)
# ============ .......................................................... ============

# Load Splitter Object
splitter = TrainValidTest_Split_Normalize(netmob_T1,dims,train_indices, valid_indices, test_indices,minmaxnorm=minmaxnorm,standardize=standardize)


# Split DataSet and Normalize accoding Stats from Training Set 
train_dataset,valid_dataset,test_dataset = splitter.load_normalize_tensor_datasets()
# Define DictDataLoader :

Time-step per hour: 4.0
coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00
Load des données NetMob .pt impossible. Création d'un random Tensor
Init NetMob Dataset:  torch.Size([7300, 40, 2, 22, 22])
Number of Nan Value:  tensor(0)
Total Number of Elements:  282656000
Tackling Training Set
reshaped_vector: [1, 1, 1, 1, 1]
repeat_vector: [1, 1, 1, 1, 1]
S size:  torch.Size([40, 2])
dims: [0, 3, 4]
conj_dims: [1, 2] 

reshaped_vector: [1, 1, 1, 1, 1]
repeat_vector: [1, 1, 1, 1, 1]
S size:  torch.Size([40, 2])
dims: [0, 3, 4]
conj_dims: [1, 2] 

Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%
Tackling Validation Set
Tackling Testing Set
reshaped_vector: [1, 1, 1, 1, 1]
repeat_vector: [1, 1, 1, 1, 1]
S size:  torch.Size([40, 2])
dims: [0, 3, 4]
conj_dims: [1, 2] 

reshaped_vector: [1, 1, 1, 1, 1]
repeat_vector: [1, 1, 1, 1, 1]
S size:  torch.Size([40, 2])
dims: [0, 3, 4]
c

In [ ]:
# Passage dans split Tensor: 
# >>>>  Consiste à Normalizer un input (Ici 'U') selon une certaine référence (ici train_input)
# >>>>  Puis de renovyer les U_train/U_valid/U_test 
# >>>>
# >>>>   Pour se faire, ils calculent d'abord les stats (min,max,mean,std) sur le tensor de référence train_input
# >>>>   Puis, à chacun des split U_Train/U_Valid/U_Test, normalisation selon les statistiques.


# ==== ATTENTION il faudra sans doute récupérer les stats simplement et pas normalizer la ref
# ==== Calcul inutile ici 

In [3]:
# ========== Comparaison avec le df_verif sur les timestamp dates:  ==========
time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding = dataset.split_normalize_load_feature_vect(invalid_dates,args.train_prop,args.valid_prop,args.test_prop,dims_agg = [0])

reshaped_vector: [1, 1]
repeat_vector: [1, 1]
S size:  torch.Size([40])
dims: [0]
conj_dims: [1] 

reshaped_vector: [1, 1]
repeat_vector: [1, 1]
S size:  torch.Size([40])
dims: [0]
conj_dims: [1] 

Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%
Tackling Training Set
reshaped_vector: [1, 1, 1]
repeat_vector: [1, 1, 1]
S size:  torch.Size([40])
dims: [0]
conj_dims: [1] 

reshaped_vector: [1, 1, 1]
repeat_vector: [1, 1, 1]
S size:  torch.Size([40])
dims: [0]
conj_dims: [1] 

Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%
Tackling Validation Set
Tackling Testing Set
reshaped_vector: [1, 1, 1]
repeat_vector: [1, 1, 1]
S size:  torch.Size([40])
dims: [0]
conj_dims: [1] 

reshaped_vector: [1, 1, 1]
repeat_vector: [1, 1, 1]
S size:  torch.Size([40])
dims: [0]
conj_dims: [1] 

Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%
reshaped_vector: [1, 1, 1]
repeat_vector: [1, 1, 1]
S size:  torch.Size([40])
dims: 

In [2]:
dataset_no_norm = dataset.clean_dataset_get_tensor_and_train_valid_test_split(dataset.df,invalid_dates,args.train_prop,args.valid_prop,args.test_prop, normalize = False)

TypeError: clean_dataset_get_tensor_and_train_valid_test_split() missing 1 required positional argument: 'tuple_agg'

In [31]:
from dataset import TensorDataset
tensor = dataset_no_norm.train_input
tensor_ds = TensorDataset(tensor,mini=None,maxi=None,mean=None,std=None, normalized = False)


In [52]:
tensor_ds.mini = reshaped_inputs.min(-1).values  
tensor_ds.maxi = reshaped_inputs.max(-1).values  
tensor_ds.mean = reshaped_inputs.mean(-1).values  
tensor_ds.std = reshaped_inputs.std(-1).values  

In [55]:
# Normalize
normalized_tensor_1 = tensor_ds.transform(reshaped_inputs,minmaxnorm,standardize,reverse=False)

# reshape-back, inverse-permute
normalized_tensor_2 = tensor_ds.inverse_reshape_permute(normalized_tensor_1)

Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%


In [43]:
normalized_tensor_ds = tensor_ds.normalize_tensor(dims = [0], minmaxnorm=True)
normalized_tensor_ds.tensor

RuntimeError: The size of tensor a (40) must match the size of tensor b (3606) at non-singleton dimension 0

In [31]:
T,N,C,H,W,L = 100,40,2,20,20,6

# Test avec subway-in
X = torch.randn(T,N,L)
I = torch.randn(T,N)
dims = [0]  # min = [N]
S = torch.randn(N)
# ...

tensor_ds = TensorDataset(X,mini=S,maxi=S*S,mean=S,std=S, normalized = False)
tensor_ds.normalize_tensor(dims, minmaxnorm= True)


# Test avec NetMob Image
X = torch.randn(T,N,C,H,W,L)
I = torch.randn(T,N,C,H,W)
dims = [0,3,4]  # min = [N]
S = torch.randn(N,C)
# ....


tensor_ds = TensorDataset(X,mini=S,maxi=S*S,mean=S,std=S, normalized = False)
tensor_ds.normalize_tensor(dims, minmaxnorm= True)

In [3]:
dataset_norm = dataset.clean_dataset_get_tensor_and_train_valid_test_split(dataset.df,invalid_dates,args.train_prop,args.valid_prop,args.test_prop, normalize = True)

Tackling Training Set


TypeError: expected Tensor as element 0 in argument 0, but got numpy.float64

In [4]:
dataset.split_normalize_load_feature_vect(args,invalid_dates,args.train_prop, args.valid_prop, args.test_prop)

Tackling Training Set


TypeError: expected Tensor as element 0 in argument 0, but got numpy.float64

## Pour le cas des NetMob Data : 

In [13]:
dataset.df_dates

,date
0,2019-03-16 00:00:00
1,2019-03-16 00:15:00
2,2019-03-16 00:30:00
3,2019-03-16 00:45:00
4,2019-03-16 01:00:00
...,...
7387,2019-05-31 22:45:00
7388,2019-05-31 23:00:00
7389,2019-05-31 23:15:00
7390,2019-05-31 23:30:00


In [14]:
dataset.df_verif

,t-672,t-96,t-6,t-5,t-4,t-3,t-2,t-1,t+0
672,2019-03-16 00:00:00,2019-03-22 00:00:00,2019-03-22 22:30:00,2019-03-22 22:45:00,2019-03-22 23:00:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00
673,2019-03-16 00:15:00,2019-03-22 00:15:00,2019-03-22 22:45:00,2019-03-22 23:00:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00
674,2019-03-16 00:30:00,2019-03-22 00:30:00,2019-03-22 23:00:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00,2019-03-23 00:30:00
675,2019-03-16 00:45:00,2019-03-22 00:45:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00,2019-03-23 00:30:00,2019-03-23 00:45:00
676,2019-03-16 01:00:00,2019-03-22 01:00:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00,2019-03-23 00:30:00,2019-03-23 00:45:00,2019-03-23 01:00:00
...,...,...,...,...,...,...,...,...,...
7387,2019-05-24 22:45:00,2019-05-30 22:45:00,2019-05-31 21:15:00,2019-05-31 21:30:00,2019-05-31 21:45:00,2019-05-31 22:00:00,2019-05-31 22:15:00,2019-05-31 22:30:00,2019-05-31 22:45:00
7388,2019-05-24 23:00:00,2019-05-30 23:00:00,2019-05-31 21:30:00,2019-05-31 21:45:00,2019-05-31 22:00:00,2019-05-31 22:15:00,2019-05-31 22:30:00,2019-05-31 22:45:00,2019-05-31 23:00:00
7389,2019-05-24 23:15:00,2019-05-30 23:15:00,2019-05-31 21:45:00,2019-05-31 22:00:00,2019-05-31 22:15:00,2019-05-31 22:30:00,2019-05-31 22:45:00,2019-05-31 23:00:00,2019-05-31 23:15:00
7390,2019-05-24 23:30:00,2019-05-30 23:30:00,2019-05-31 22:00:00,2019-05-31 22:15:00,2019-05-31 22:30:00,2019-05-31 22:45:00,2019-05-31 23:00:00,2019-05-31 23:15:00,2019-05-31 23:30:00
